In [2]:
import pandas as pd 
data_path = '../input/'

In [3]:
train = pd.read_pickle(data_path+'/train.pkl')

In [4]:
train.shape

(24236806, 10)

In [5]:
train.groupby('Asset_ID')['Close'].shift(15)

0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
               ...     
24236801     156.098857
24236802    2409.222800
24236803       0.090970
24236804       0.280169
24236805     231.680000
Name: Close, Length: 24236806, dtype: float64

In [6]:
s_train = pd.read_csv(data_path+'g-research-crypto-forecasting/supplemental_train.csv')
s_train.to_pickle(data_path+'supplemental_train.pkl')

In [7]:
s_train.head(15)

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1623542400,3,1201.0,1.478556,1.486030,1.478000,1.483681,6.547996e+05,1.481439,-0.002594
1,1623542400,2,1020.0,580.306667,583.890000,579.910000,582.276667,1.227988e+03,581.697038,-0.009143
2,1623542400,0,626.0,343.789500,345.108000,343.640000,344.598000,1.718833e+03,344.441729,-0.004525
3,1623542400,1,2888.0,35554.289632,35652.464650,35502.670000,35602.004286,1.638115e+02,35583.469303,0.003096
4,1623542400,4,433.0,0.312167,0.312600,0.311920,0.312208,5.855774e+05,0.312154,0.001426
5,1623542400,5,359.0,4.832550,4.845900,4.822900,4.837583,4.714355e+04,4.836607,-0.000579
6,1623542400,7,541.0,55.223080,55.494000,55.182000,55.344680,6.625202e+03,55.298816,-0.003998
7,1623542400,6,2186.0,2371.194286,2379.200000,2369.670000,2374.380714,1.214129e+03,2374.335307,0.002565
8,1623542400,8,35.0,1.003150,1.019800,0.987300,1.003300,7.061928e+03,1.002936,-0.005097
9,1623542400,9,560.0,161.933429,162.480000,161.730000,162.214714,1.485009e+03,162.231310,0.000686


In [8]:
s_train.shape

(2015112, 10)

In [9]:
am = s_train.groupby('Asset_ID').Close.transform(lambda x: x.rolling(window=5).mean())
am.tail(5)

2015107     156.809571
2015108    2426.988343
2015109       0.091123
2015110       0.281477
2015111     231.791100
Name: Close, dtype: float64

In [10]:
s_train.groupby('Asset_ID').Close.shift(1).head(20)

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
5              NaN
6              NaN
7              NaN
8              NaN
9              NaN
10             NaN
11             NaN
12             NaN
13             NaN
14        1.483681
15      582.276667
16      344.598000
17    35602.004286
18        0.312208
19        4.837583
Name: Close, dtype: float64

In [11]:
import numpy as np
def _rsiFunc(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi

In [12]:
s_train.groupby('Asset_ID').Close.transform(lambda x: _rsiFunc(x.values, 5))

0          18.724013
1          22.518331
2          30.796164
3          30.320739
4          17.173412
             ...    
2015107    49.859647
2015108    57.611476
2015109    58.297421
2015110    55.637040
2015111    55.028428
Name: Close, Length: 2015112, dtype: float64